In [11]:
import os
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report
)

In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Usando dispositivo: {device}")

Usando dispositivo: cuda


In [13]:
config = {
        'batch_size': 64,
        'num_epochs': 100,
        'output_dir': 'mlp_results_opt',
        'use_augmentation': True  #Cambiar a False si no se quiere augmentation
    }

print(f"\nConfiguración:")
for key, value in config.items():
    print(f"   {key}: {value}")


Configuración:
   batch_size: 64
   num_epochs: 100
   output_dir: mlp_results_opt
   use_augmentation: True


## MLP SIN FEATURE EXTRACTOR

In [14]:
def load_raw_pixel_data(prefix='covid19_raw_opt'):
    """Carga datos de píxeles crudos optimizados"""
    print(f"\nCargando datos RAW")
    
    X_train = np.load(f'{prefix}_X_train.npy')
    X_val = np.load(f'{prefix}_X_val.npy')
    X_test = np.load(f'{prefix}_X_test.npy')
    y_train = np.load(f'{prefix}_y_train.npy')
    y_val = np.load(f'{prefix}_y_val.npy')
    y_test = np.load(f'{prefix}_y_test.npy')
    
    print(f"   Train: {X_train.shape}")
    print(f"   Val:   {X_val.shape}")
    print(f"   Test:  {X_test.shape}")
    
    return X_train, X_val, X_test, y_train, y_val, y_test

In [15]:
X_train_raw, X_val_raw, X_test_raw, y_train_raw, y_val_raw, y_test_raw = \
        load_raw_pixel_data('covid19_raw_opt')


Cargando datos RAW
   Train: (28536, 16384)
   Val:   (6116, 16384)
   Test:  (6116, 16384)


In [16]:
def prepare_dataloaders(X_train, X_val, X_test, y_train, y_val, y_test, batch_size=64):
    """Prepara DataLoaders de PyTorch"""
    le = LabelEncoder()
    y_train_encoded = le.fit_transform(y_train)
    y_val_encoded = le.transform(y_val)
    y_test_encoded = le.transform(y_test)
    
    X_train_tensor = torch.FloatTensor(X_train)
    X_val_tensor = torch.FloatTensor(X_val)
    X_test_tensor = torch.FloatTensor(X_test)
    
    y_train_tensor = torch.LongTensor(y_train_encoded)
    y_val_tensor = torch.LongTensor(y_val_encoded)
    y_test_tensor = torch.LongTensor(y_test_encoded)
    
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
    test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
    print(f"\nDataLoaders creados (batch_size={batch_size})")
    print(f"   Train batches: {len(train_loader)}")
    print(f"   Val batches:   {len(val_loader)}")
    print(f"   Test batches:  {len(test_loader)}")
    
    return train_loader, val_loader, test_loader, le

In [17]:
class MLP_Shallow(nn.Module):
    """
    Arquitectura Shallow
    """
    def __init__(self, input_size, num_classes):
        super(MLP_Shallow, self).__init__()
        
        self.network = nn.Sequential(
            nn.Linear(input_size, 384),
            nn.BatchNorm1d(384),
            nn.ReLU(),
            nn.Dropout(0.4),
            
            nn.Linear(384, 192),
            nn.BatchNorm1d(192),
            nn.ReLU(),
            nn.Dropout(0.4),
            
            nn.Linear(192, num_classes)
        )
        
    def forward(self, x):
        return self.network(x)

In [18]:
class MLP_Medium(nn.Module):
    """
    Arquitectura Medium
    """
    def __init__(self, input_size, num_classes):
        super(MLP_Medium, self).__init__()
        
        self.network = nn.Sequential(
            nn.Linear(input_size, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.5),
            
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.5),
            
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.4),
            
            nn.Linear(128, num_classes)
        )
        
    def forward(self, x):
        return self.network(x)

In [19]:
class MLP_Deep_Regularized(nn.Module):
    """
    Arquitectura Deep 
    """
    def __init__(self, input_size, num_classes):
        super(MLP_Deep_Regularized, self).__init__()
        
        self.network = nn.Sequential(
            nn.Linear(input_size, 768),
            nn.BatchNorm1d(768),
            nn.ReLU(),
            nn.Dropout(0.6),
            
            nn.Linear(768, 384),
            nn.BatchNorm1d(384),
            nn.ReLU(),
            nn.Dropout(0.5),
            
            nn.Linear(384, 192),
            nn.BatchNorm1d(192),
            nn.ReLU(),
            nn.Dropout(0.5),
            
            nn.Linear(192, 96),
            nn.BatchNorm1d(96),
            nn.ReLU(),
            nn.Dropout(0.4),
            
            nn.Linear(96, num_classes)
        )
        
    def forward(self, x):
        return self.network(x)

In [20]:
def validate(model, val_loader, criterion, device):
    """Valida el modelo"""
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    val_loss = running_loss / total
    val_acc = correct / total
    
    return val_loss, val_acc

In [21]:
def get_augmented_batch(X_batch, img_size=128, augmentation_probability=0.3):
    """
    Aplica data augmentation on-the-fly durante entrenamiento
    
    Solo para datos RAW (píxeles), no para LBP
    """
    X_aug = X_batch.clone()
    batch_size, features = X_aug.shape
    
    # Verificar que el tamaño es correcto
    if features != img_size * img_size:
        return X_aug  # No augmentar si no es una imagen cuadrada
    
    for i in range(batch_size):
        if np.random.rand() < augmentation_probability:
            # Reshape a imagen
            img = X_aug[i].reshape(img_size, img_size).cpu().numpy()
            
            # Aplicar transformación aleatoria
            aug_type = np.random.choice([
                'rotate', 'flip', 'shift', 'noise', 'brightness'
            ])
            
            if aug_type == 'rotate':
                angle = np.random.uniform(-10, 10)
                M = cv2.getRotationMatrix2D((img_size/2, img_size/2), angle, 1.0)
                img = cv2.warpAffine(img, M, (img_size, img_size), borderMode=cv2.BORDER_REFLECT)
            
            elif aug_type == 'flip':
                img = np.fliplr(img)
            
            elif aug_type == 'shift':
                shift_x = int(np.random.uniform(-5, 5))
                shift_y = int(np.random.uniform(-5, 5))
                M = np.float32([[1, 0, shift_x], [0, 1, shift_y]])
                img = cv2.warpAffine(img, M, (img_size, img_size), borderMode=cv2.BORDER_REFLECT)
            
            elif aug_type == 'noise':
                noise = np.random.normal(0, 0.05, img.shape)
                img = img + noise
            
            elif aug_type == 'brightness':
                factor = np.random.uniform(0.9, 1.1)
                img = img * factor
            
            # Volver a aplanar
            X_aug[i] = torch.from_numpy(img.flatten().astype(np.float32))
    
    return X_aug

In [ ]:
def train_epoch(model, train_loader, criterion, optimizer, device, 
                         use_augmentation=False, img_size=128):
    """
    Entrena el modelo por una época
    """
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for inputs, labels in train_loader:
        # Aplicar augmentation si está habilitado
        if use_augmentation:
            inputs = get_augmented_batch(inputs, img_size, augmentation_probability=0.3)
        
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        
        # Gradient clipping para estabilidad
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        
        optimizer.step()
        
        running_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    epoch_loss = running_loss / total
    epoch_acc = correct / total
    
    return epoch_loss, epoch_acc


In [23]:
def train_model(model, train_loader, val_loader, num_epochs, device, 
                         model_name, use_augmentation=False, img_size=128):
    """
    Entrena el modelo
    """
    print(f"\nEntrenando {model_name}")
    
    # Criterio con label smoothing
    criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
    
    # AdamW con weight decay más fuerte
    optimizer = optim.AdamW(
        model.parameters(), 
        lr=0.001,
        weight_decay=1e-4  # Era 1e-5
    )
    
    # Learning rate scheduler: Reduce on plateau
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer,
        mode='max',
        factor=0.5,
        patience=5
    )
    
    best_val_acc = 0.0
    best_model_state = None
    patience_counter = 0
    patience = 15
    
    history = {
        'train_loss': [],
        'train_acc': [],
        'val_loss': [],
        'val_acc': [],
        'lr': []
    }
    
    start_time = time.time()
    
    for epoch in range(num_epochs):
        # Train
        train_loss, train_acc = train_epoch(
            model, train_loader, criterion, optimizer, device,
            use_augmentation, img_size
        )
        
        # Validate
        val_loss, val_acc = validate(model, val_loader, criterion, device)
        
        # Update learning rate
        scheduler.step(val_acc)
        
        # Guardar historial
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)
        history['lr'].append(optimizer.param_groups[0]['lr'])
        
        # Early stopping
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model_state = model.state_dict().copy()
            patience_counter = 0
        else:
            patience_counter += 1
        
        # Print cada 5 épocas
        if (epoch + 1) % 5 == 0:
            print(f"Epoch [{epoch+1:3d}/{num_epochs}] | "
                  f"Train: {train_loss:.4f}/{train_acc:.4f} | "
                  f"Val: {val_loss:.4f}/{val_acc:.4f} | "
                  f"LR: {optimizer.param_groups[0]['lr']:.6f}")
        
        if patience_counter >= patience:
            print(f"\nEarly stopping en época {epoch+1}")
            break
    
    training_time = time.time() - start_time
    model.load_state_dict(best_model_state)
    
    print(f"\n✓ Entrenamiento completado en {training_time:.2f}s")
    print(f"  Mejor Val Accuracy: {best_val_acc:.4f}")
    print("="*70)
    
    return model, history, training_time, best_val_acc

In [24]:
def evaluate_model(model, test_loader, device, label_encoder):
    """Evalúa el modelo en el conjunto de test"""
    model.eval()
    all_preds = []
    all_labels = []
    
    inference_times = []
    
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            
            start_time = time.time()
            outputs = model(inputs)
            inference_time = time.time() - start_time
            inference_times.append(inference_time)
            
            _, predicted = torch.max(outputs, 1)
            
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.numpy())
    
    all_preds = label_encoder.inverse_transform(all_preds)
    all_labels = label_encoder.inverse_transform(all_labels)
    
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='weighted', zero_division=0)
    recall = recall_score(all_labels, all_preds, average='weighted', zero_division=0)
    f1 = f1_score(all_labels, all_preds, average='weighted', zero_division=0)
    
    precision_per_class = precision_score(all_labels, all_preds, average=None, zero_division=0)
    recall_per_class = recall_score(all_labels, all_preds, average=None, zero_division=0)
    f1_per_class = f1_score(all_labels, all_preds, average=None, zero_division=0)
    
    cm = confusion_matrix(all_labels, all_preds, labels=label_encoder.classes_)
    
    avg_inference_time = np.mean(inference_times) * 1000
    
    metrics = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'precision_per_class': precision_per_class,
        'recall_per_class': recall_per_class,
        'f1_per_class': f1_per_class,
        'confusion_matrix': cm,
        'avg_inference_time_ms': avg_inference_time,
        'predictions': all_preds,
        'true_labels': all_labels
    }
    
    return metrics


In [25]:
def print_metrics(metrics, class_names, model_name):
    """Imprime métricas de forma legible"""
    print(f"\nMétricas de {model_name}")
    print(f"Accuracy:  {metrics['accuracy']:.4f}")
    print(f"Precision: {metrics['precision']:.4f}")
    print(f"Recall:    {metrics['recall']:.4f}")
    print(f"F1-Score:  {metrics['f1']:.4f}")
    print(f"Inference: {metrics['avg_inference_time_ms']:.2f} ms/batch")
    
    print(f"\nMétricas por clase:")
    print(f"{'Clase':<25} {'Precision':<12} {'Recall':<12} {'F1-Score':<12}")
    
    for i, class_name in enumerate(class_names):
        print(f"{class_name:<25} "
              f"{metrics['precision_per_class'][i]:<12.4f} "
              f"{metrics['recall_per_class'][i]:<12.4f} "
              f"{metrics['f1_per_class'][i]:<12.4f}")
    

In [26]:
def plot_training_history(history, model_name, save_path):
    """Grafica el historial de entrenamiento"""
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    epochs = range(1, len(history['train_loss']) + 1)
    
    # Loss
    axes[0].plot(epochs, history['train_loss'], 'b-', label='Train Loss', linewidth=2)
    axes[0].plot(epochs, history['val_loss'], 'r-', label='Val Loss', linewidth=2)
    axes[0].set_xlabel('Época', fontsize=12)
    axes[0].set_ylabel('Loss', fontsize=12)
    axes[0].set_title(f'{model_name} - Loss', fontsize=14, fontweight='bold')
    axes[0].legend(fontsize=10)
    axes[0].grid(True, alpha=0.3)
    
    # Accuracy
    axes[1].plot(epochs, history['train_acc'], 'b-', label='Train Acc', linewidth=2)
    axes[1].plot(epochs, history['val_acc'], 'r-', label='Val Acc', linewidth=2)
    axes[1].set_xlabel('Época', fontsize=12)
    axes[1].set_ylabel('Accuracy', fontsize=12)
    axes[1].set_title(f'{model_name} - Accuracy', fontsize=14, fontweight='bold')
    axes[1].legend(fontsize=10)
    axes[1].grid(True, alpha=0.3)
    
    # Learning Rate
    axes[2].plot(epochs, history['lr'], 'g-', linewidth=2)
    axes[2].set_xlabel('Época', fontsize=12)
    axes[2].set_ylabel('Learning Rate', fontsize=12)
    axes[2].set_title(f'{model_name} - Learning Rate', fontsize=14, fontweight='bold')
    axes[2].set_yscale('log')
    axes[2].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(save_path, dpi=150, bbox_inches='tight')
    plt.close()
    
    print(f"Gráfica guardada: {save_path}")


def plot_confusion_matrix(cm, class_names, model_name, save_path):
    """Grafica la matriz de confusión"""
    plt.figure(figsize=(10, 8))
    
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    
    sns.heatmap(cm_normalized, annot=True, fmt='.2f', cmap='Blues',
                xticklabels=class_names, yticklabels=class_names,
                cbar_kws={'label': 'Proporción'})
    
    plt.title(f'{model_name} - Matriz de Confusión (Normalizada)', 
              fontsize=14, fontweight='bold', pad=20)
    plt.ylabel('Etiqueta Real', fontsize=12)
    plt.xlabel('Etiqueta Predicha', fontsize=12)
    plt.xticks(rotation=45, ha='right')
    plt.yticks(rotation=0)
    
    plt.tight_layout()
    plt.savefig(save_path, dpi=150, bbox_inches='tight')
    plt.close()
    
    print(f"Matriz de confusión guardada: {save_path}")


def plot_comparison_table(results_df, save_path):
    """Crea tabla comparativa de todos los modelos"""
    fig, ax = plt.subplots(figsize=(16, len(results_df) * 0.8))
    ax.axis('tight')
    ax.axis('off')
    
    table_data = []
    for _, row in results_df.iterrows():
        table_data.append([
            row['Modelo'],
            f"{row['Accuracy']:.4f}",
            f"{row['Precision']:.4f}",
            f"{row['Recall']:.4f}",
            f"{row['F1-Score']:.4f}",
            f"{row['Train Time (s)']:.2f}",
            f"{row['Inference (ms)']:.2f}",
            f"{row['Params']:,}"
        ])
    
    columns = ['Modelo', 'Accuracy', 'Precision', 'Recall', 'F1-Score', 
               'Train Time (s)', 'Inference (ms)', 'Parámetros']
    
    table = ax.table(cellText=table_data, colLabels=columns,
                     cellLoc='center', loc='center',
                     colWidths=[0.20, 0.10, 0.10, 0.10, 0.10, 0.12, 0.12, 0.12])
    
    table.auto_set_font_size(False)
    table.set_fontsize(9)
    table.scale(1, 2)
    
    for i in range(len(columns)):
        table[(0, i)].set_facecolor('#4472C4')
        table[(0, i)].set_text_props(weight='bold', color='white')
    
    for i in range(1, len(table_data) + 1):
        if i % 2 == 0:
            for j in range(len(columns)):
                table[(i, j)].set_facecolor('#F0F0F0')
    
    plt.title('Comparación de Modelos MLP OPTIMIZADOS', fontsize=16, fontweight='bold', pad=20)
    plt.savefig(save_path, dpi=150, bbox_inches='tight')
    plt.close()
    
    print(f"Tabla comparativa guardada: {save_path}")

In [27]:
def train_all_architectures(X_train, X_val, X_test, y_train, y_val, y_test,
                                     data_type, batch_size=64, num_epochs=100, 
                                     output_dir='results_opt', use_augmentation=False):
    """
    Entrena todas las arquitecturas OPTIMIZADAS
    """
    print(f"Entrenando modelos MLP - {data_type.upper()}")
    
    output_path = Path(output_dir) / data_type
    output_path.mkdir(parents=True, exist_ok=True)
    
    train_loader, val_loader, test_loader, le = prepare_dataloaders(
        X_train, X_val, X_test, y_train, y_val, y_test, batch_size
    )
    
    input_size = X_train.shape[1]
    num_classes = len(le.classes_)
    class_names = le.classes_
    
    # Determinar img_size para augmentation
    img_size = int(np.sqrt(input_size)) if input_size == 128*128 else 0
    
    print(f"\nConfiguración:")
    print(f"   Input size: {input_size}")
    print(f"   Num classes: {num_classes}")
    print(f"   Classes: {list(class_names)}")
    print(f"   Data augmentation: {'Enabled' if use_augmentation and img_size > 0 else '✗ Disabled'}")
    
    # Definir arquitecturas OPTIMIZADAS
    architectures = {
        f'{data_type}_shallow': MLP_Shallow(input_size, num_classes),
        f'{data_type}_medium': MLP_Medium(input_size, num_classes),
        f'{data_type}_deep_regularized': MLP_Deep_Regularized(input_size, num_classes)
    }
    
    results = []
    
    for model_name, model in architectures.items():
        print(f"Arquitectura: {model_name}")
        
        model = model.to(device)
        
        num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
        print(f"   Parámetros entrenables: {num_params:,}")
        
        # Entrenar con optimizaciones
        model, history, train_time, best_val_acc = train_model(
            model, train_loader, val_loader, num_epochs, device, model_name,
            use_augmentation=(use_augmentation and img_size > 0), img_size=img_size
        )
        
        # Evaluar
        metrics = evaluate_model(model, test_loader, device, le)
        
        print_metrics(metrics, class_names, model_name)
        
        # Guardar modelo
        model_path = output_path / f'{model_name}.pth'
        torch.save({
            'model_state_dict': model.state_dict(),
            'input_size': input_size,
            'num_classes': num_classes,
            'class_names': list(class_names),
            'label_encoder': le,
            'metrics': metrics,
            'history': history
        }, model_path)
        print(f"\nModelo guardado: {model_path}")
        
        # Visualizaciones
        plot_training_history(history, model_name, 
                            output_path / f'{model_name}_history.png')
        plot_confusion_matrix(metrics['confusion_matrix'], class_names, model_name,
                            output_path / f'{model_name}_confusion.png')
        
        # Guardar resultados
        results.append({
            'Modelo': model_name,
            'Accuracy': metrics['accuracy'],
            'Precision': metrics['precision'],
            'Recall': metrics['recall'],
            'F1-Score': metrics['f1'],
            'Train Time (s)': train_time,
            'Inference (ms)': metrics['avg_inference_time_ms'],
            'Params': num_params,
            'Best Val Acc': best_val_acc
        })
    
    results_df = pd.DataFrame(results)
    results_df = results_df.sort_values('Accuracy', ascending=False)
    
    csv_path = output_path / 'comparison.csv'
    results_df.to_csv(csv_path, index=False)
    print(f"\nResultados guardados: {csv_path}")
    
    plot_comparison_table(results_df, output_path / 'comparison_table.png')
    
    print(f"RESUMEN - {data_type.upper()}")
    print(results_df.to_string(index=False))
    
    return results_df

In [28]:
results_raw = train_all_architectures(
        X_train_raw, X_val_raw, X_test_raw,
        y_train_raw, y_val_raw, y_test_raw,
        data_type='raw',
        batch_size=config['batch_size'],
        num_epochs=config['num_epochs'],
        output_dir=config['output_dir'],
        use_augmentation=config['use_augmentation']
    )

Entrenando modelos MLP - RAW

DataLoaders creados (batch_size=64)
   Train batches: 446
   Val batches:   96
   Test batches:  96

Configuración:
   Input size: 16384
   Num classes: 4
   Classes: [np.str_('COVID'), np.str_('Lung_Opacity'), np.str_('Normal'), np.str_('Viral Pneumonia')]
   Data augmentation: Enabled
Arquitectura: raw_shallow
   Parámetros entrenables: 6,367,684

Entrenando raw_shallow
Epoch [  5/100] | Train: 0.6822/0.8275 | Val: 0.6545/0.8396 | LR: 0.001000
Epoch [ 10/100] | Train: 0.6179/0.8675 | Val: 0.6215/0.8592 | LR: 0.001000
Epoch [ 15/100] | Train: 0.5754/0.8928 | Val: 0.5926/0.8733 | LR: 0.001000
Epoch [ 20/100] | Train: 0.5425/0.9158 | Val: 0.5855/0.8818 | LR: 0.001000
Epoch [ 25/100] | Train: 0.5229/0.9261 | Val: 0.5774/0.8887 | LR: 0.001000
Epoch [ 30/100] | Train: 0.5049/0.9363 | Val: 0.5752/0.8896 | LR: 0.001000
Epoch [ 35/100] | Train: 0.4994/0.9400 | Val: 0.5706/0.8900 | LR: 0.000500
Epoch [ 40/100] | Train: 0.4726/0.9564 | Val: 0.5673/0.8932 | LR: 0.00